In [43]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image

from model import build_model
from load_data import get_patches, data_generate
from sklearn.model_selection import train_test_split

PATCH_SHAPE = (224, 224, 3)

clinical_data = pd.read_csv("data/train.csv")

bag_names = list(clinical_data["ID"])
labels = list(clinical_data["N_category"])
patch_bags = get_patches("data/train_imgs_patch")

In [45]:
train_bag_names, val_bag_names, train_y, val_y, train_bags, val_bags = train_test_split(bag_names, labels, patch_bags)

In [46]:
train_dataset = tf.data.Dataset.from_generator(
    generator=data_generate,
    output_types=(tf.float32, tf.float32),
    output_shapes=(tf.TensorShape([None, 224, 224, 3]), tf.TensorShape([1, ])),
    args=(train_bag_names, train_y, train_bags)
)

val_dataset = tf.data.Dataset.from_generator(
    generator=data_generate,
    output_types=(tf.float32, tf.float32),
    output_shapes=(tf.TensorShape([None, 224, 224, 3]), tf.TensorShape([1, ])),
    args=(val_bag_names, val_y, val_bags)
)

In [47]:
bag_img_tensor, bag_label_tensor = next(iter(train_dataset))

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} TypeError: `generator` yielded an element of shape (1,) where an element of shape () was expected.
Traceback (most recent call last):

  File "c:\Users\4pygm\miniconda3\envs\breast\lib\site-packages\tensorflow\python\ops\script_ops.py", line 271, in __call__
    ret = func(*args)

  File "c:\Users\4pygm\miniconda3\envs\breast\lib\site-packages\tensorflow\python\autograph\impl\api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "c:\Users\4pygm\miniconda3\envs\breast\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py", line 1079, in generator_py_func
    f"`generator` yielded an element of shape {ret_array.shape} "

TypeError: `generator` yielded an element of shape (1,) where an element of shape () was expected.


	 [[{{node PyFunc}}]] [Op:IteratorGetNext]

In [ ]:
bag_img_tensor.shape, bag_label_tensor.shape

In [ ]:
model = build_model(PATCH_SHAPE)
model.summary()

In [ ]:
from loss import bag_loss, bag_accuracy
model.compile(
    optimizer=tf.keras.optimizers.Adam(), loss=bag_loss, metrics=[bag_accuracy]
)

In [ ]:
model.fit(
    train_dataset,
    validation_data=val_dataset
)